In [2]:
import sympy
from sympy import Derivative, Function, Symbol, symbols, Eq, pi, cos, sin, exp, log, oo
from sympy import Function, dsolve, Derivative, checkodesol
from sympy import fps, Rational
from sympy import pprint, Matrix, eye, zeros
from sympy import Inverse
from utils import *
z = Symbol('z')

Restate the problem that we want to solve: 

$$z^k \frac{d\psi}{dz} = A(z)\psi$$, where $\psi$ is a vector-valued function and $A(z)$ as a given matrix-valued function is holomorphic.

In the general case where we can't find it directly, we use Formal Gauge transform to simplify the system, and solve it in diagonal version. Then, $\hat{\Phi}(u, z) = \hat{\Psi}(t(u, z)) \hat{\Psi}^{-1}(s(u,z))$ is the formal universal solution on Sto_k, while since $\Phi$ is unique, then this means that $\hat{\Phi}$ is the power series of $\Phi$, so it must be convergent since $\Phi$ is entire.

To do the Gauge transformation, the core idea is to transfer it to the case where $z^k \frac{d\phi}{dz} = (D_0+zD_1+...z^{k-1}D_{k-1})\phi$, where $D_i$ are diagonal, which can be easily solve by entries on both sides.

In order to solve this problem, we first need to make a change of variables $\psi' = F \psi$, whereas $F = 1+zF_1 + z^2F_2 + ...$









In this step, we let $F = (I+zH_1)(I+z^2H_2)...$

where $H_p = \begin{cases} {ad}^{-1}_{A_0}(A_p^{OD}) \text{   if   } k > 1 \\ ({ad}_{A_0} - p)^{-1}(A_p^{OD}) \text{    if   } k = 1  \end{cases}$

After this transform, the ODE becomes $z^k \frac{d\psi'}{dz} = (diagonal(A_0) + diagonal(A_1)z + ...) \psi'$

After doing this, we make the second Gauge Transform to transform it to the finite case, 

which is the transform $K = \exp(-\int{D_k+D_{k+1}z+...})$


Finally, the simplified system is $z^k \frac{d\psi''}{dz} = (diagonal(A_0) + diagonal(A_1)z + ...+ diagonal(A_{k-1})z^{k-1}) \psi''$

and the overall Gauge transform is $(KF)$[...], and the inverse is $(KF)^{-1}$[...], so $\psi = (KF)^{-1}\psi''$

After we get the fundamental solution $\Psi$, we can construct $\Phi$ on the Sto_k in a similar way.

Example1: 

$z^2 \psi' = \begin{pmatrix} \cos(z)+1 & \sin(z) \\ -\sin(z) & \cos(z) \end{pmatrix} \psi$ 

We are working on Sto_2

We trancate the Gauge Transform series up to some given order 

In [67]:
A = Matrix([[cos(z)+1, sin(z)], [-sin(z), cos(z)]])
n = 2
k = 2
order = 3
# This means that we only approaching using the first 3 terms

In [10]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [11]:
K_trunc

Matrix([
[cosh(z**2/12), sinh(z**2/12)],
[sinh(z**2/12), cosh(z**2/12)]])

In [12]:
F_trunc

Matrix([
[1, z],
[z, 1]])

In [16]:
Total

Matrix([
[z*sinh(z**2/12) + cosh(z**2/12), z*cosh(z**2/12) + sinh(z**2/12)],
[z*cosh(z**2/12) + sinh(z**2/12), z*sinh(z**2/12) + cosh(z**2/12)]])

In [18]:
Total_inverse = Total.inverse()

In [19]:
Total_inverse

Matrix([
[(-z*sinh(z**2/12) - cosh(z**2/12))/(-z**2*sinh(z**2/12)**2 + z**2*cosh(z**2/12)**2 + sinh(z**2/12)**2 - cosh(z**2/12)**2),  (z*cosh(z**2/12) + sinh(z**2/12))/(-z**2*sinh(z**2/12)**2 + z**2*cosh(z**2/12)**2 + sinh(z**2/12)**2 - cosh(z**2/12)**2)],
[ (z*cosh(z**2/12) + sinh(z**2/12))/(-z**2*sinh(z**2/12)**2 + z**2*cosh(z**2/12)**2 + sinh(z**2/12)**2 - cosh(z**2/12)**2), (-z*sinh(z**2/12) - cosh(z**2/12))/(-z**2*sinh(z**2/12)**2 + z**2*cosh(z**2/12)**2 + sinh(z**2/12)**2 - cosh(z**2/12)**2)]])

Note that Gauge Transform is a group action, meaning that $F_1[F_2[A]] = (F_1F_2)[A]$ so the inverse Gauge Transform is the Gauge Transform of the inverse Matrix, and composed Gauge Transform is the transform of the product Matrix.

Finally, the equation is $z^k \psi' = diag(A_0) + diag(A_1)z + ... + diag(A_{k-1})z^{k-1}$

In [22]:
Psi_prime, Phi_prime = solve_phi(k, A)

In [23]:
Psi_prime

Matrix([
[exp(-2/z),         0],
[        0, exp(-1/z)]])

In [24]:
Phi_prime

Matrix([
[exp(2*(1 - exp(-u*z))/z),                      0],
[                       0, exp((1 - exp(-u*z))/z)]])

The final solution is $T^{-1}\Psi$ (we use $T$ to denote the initial change of coordinate that we applied to simplify the system)

$T^{-1}$ is the Total_inverse that we just found above.

So the final solution $\Psi$ is 

In [28]:
Psi = Total_inverse * Psi_prime

Hence, the original universal solution in the Stokes groupoid of order $k$ is 

In [31]:
Phi = get_phi_from_psi(Psi, k)

In [33]:
Phi

Matrix([
[-((z*sinh(z**2/12) + cosh(z**2/12))*(z*exp(u*z)*sinh(z**2*exp(2*u*z)/12) + cosh(z**2*exp(2*u*z)/12))*exp(((2*exp(u*z) + 1)*exp(-u*z) + 2)/z) - (z*cosh(z**2/12) + sinh(z**2/12))*(z*exp(u*z)*cosh(z**2*exp(2*u*z)/12) + sinh(z**2*exp(2*u*z)/12))*exp((3*exp(u*z) + 2)*exp(-u*z)/z))*exp(-((5*exp(u*z) + 3)*exp(-u*z) - 3)/z)/(z**2*exp(2*u*z) - 1),  ((z*sinh(z**2/12) + cosh(z**2/12))*(z*exp(u*z)*cosh(z**2*exp(2*u*z)/12) + sinh(z**2*exp(2*u*z)/12))*exp((3*exp(u*z) + 2)*exp(-u*z)/z) - (z*cosh(z**2/12) + sinh(z**2/12))*(z*exp(u*z)*sinh(z**2*exp(2*u*z)/12) + cosh(z**2*exp(2*u*z)/12))*exp(((2*exp(u*z) + 1)*exp(-u*z) + 2)/z))*exp(-((5*exp(u*z) + 3)*exp(-u*z) - 3)/z)/(z**2*exp(2*u*z) - 1)],
[ ((z*sinh(z**2/12) + cosh(z**2/12))*(z*exp(u*z)*cosh(z**2*exp(2*u*z)/12) + sinh(z**2*exp(2*u*z)/12))*exp(((2*exp(u*z) + 1)*exp(-u*z) + 2)/z) - (z*cosh(z**2/12) + sinh(z**2/12))*(z*exp(u*z)*sinh(z**2*exp(2*u*z)/12) + cosh(z**2*exp(2*u*z)/12))*exp((3*exp(u*z) + 2)*exp(-u*z)/z))*exp(-((5*exp(u*z) + 3)*exp(-u

Example2: 

$z^2 \psi' = \begin{pmatrix} z^4+3z^3+1 & z \\ z^2 & z^3+5z^2+7 \end{pmatrix} \psi$ 

We are working on Sto_2

We trancate the Gauge Transform series up to some given order 

In [35]:
A = Matrix([[z**4+3*z**3+1, z], [z**2, z**3+5*z**2+7]])
n = 2
k = 2
order = 10

In [37]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [38]:
Total

Matrix([
[                    (36 - z**3)*exp(z**2/72)/36,  -z*exp(z**2/72)/6],
[(z**3 - 2*(1 - exp(z**2/72))*(z**3 - 36))/(6*z), 2*exp(z**2/72) - 1]])

In [41]:
Total_inverse = Total.inverse()

In [43]:
Total_inverse

Matrix([
[              (-exp(z**2/72) + 2*exp(z**2/36))*exp(-z**2/36),         z/6],
[(-2*z**3 + z**3*exp(-z**2/72) + 72 - 72*exp(-z**2/72))/(6*z), 1 - z**3/36]])

Note that Gauge Transform is a group action, meaning that $F_1[F_2[A]] = (F_1F_2)[A]$ so the inverse Gauge Transform is the Gauge Transform of the inverse Matrix, and composed Gauge Transform is the transform of the product Matrix.

Finally, the equation is $z^k \psi' = diag(A_0) + diag(A_1)z + ... + diag(A_{k-1})z^{k-1}$

In [52]:
Psi_prime, Phi_prime = solve_phi(k, A)

In [53]:
Psi_prime

Matrix([
[exp(-1/z),         0],
[        0, exp(-7/z)]])

In [56]:
Phi_prime

Matrix([
[exp((1 - exp(-u*z))/z),                        0],
[                     0, exp(7*(1 - exp(-u*z))/z)]])

The final solution is $T^{-1}\Psi$ (we use $T$ to denote the change of coordinate that we applied to simplify the system)

$T^{-1}$ is the Total_inverse that we just found above.

In [58]:
T_inverse_Psi = Total_inverse * Psi_prime

In [60]:
T_inverse_Psi

Matrix([
[              (-exp(z**2/72) + 2*exp(z**2/36))*exp(-1/z)*exp(-z**2/36),           z*exp(-7/z)/6],
[(-2*z**3 + z**3*exp(-z**2/72) + 72 - 72*exp(-z**2/72))*exp(-1/z)/(6*z), (1 - z**3/36)*exp(-7/z)]])

In [62]:
Final_Phi = get_phi_from_psi(T_inverse_Psi, k)
## This will take much long time to run.

In [63]:
Final_Phi

Matrix([
[                                                      ((exp(z**2*exp(2*u*z)/72) - 2*exp(z**2*exp(2*u*z)/36))*(4*(36 - z**3)*exp(z**2/36) + (z**3 - 36)*exp(z**2/72) + 4*(z**3 - 36)*exp(z**2/24))*exp(z**2/24 + 3/z + 7*exp(-u*z)/z) + (exp(z**2/24 + 2/z) - 4*exp(z**2/18 + 2/z) + 4*exp(5*z**2/72 + 2/z))*(-z**3*exp(7/z) + 2*z**3*exp(z**2/72 + 7/z) + 72*exp(7/z) - 72*exp(z**2/72 + 7/z))*exp(u*z + z**2*exp(2*u*z)/36 + exp(-u*z)/z))*exp(-z**2*exp(2*u*z)/36 - 2/z - 8*exp(-u*z)/z)/(36*(4*exp(5*z**2/72) + exp(z**2/24) - 4*exp(z**2/18))),                                                   z*((-exp(7/z) + 2*exp(z**2/72 + 7/z))*(exp(z**2/24 + 2/z) - 4*exp(z**2/18 + 2/z) + 4*exp(5*z**2/72 + 2/z))*exp(u*z + z**2*exp(2*u*z)/36 + exp(-u*z)/z) + (exp(z**2*exp(2*u*z)/72) - 2*exp(z**2*exp(2*u*z)/36))*(exp(z**2/72) - 4*exp(z**2/36) + 4*exp(z**2/24))*exp(z**2/24 + 3/z + 7*exp(-u*z)/z))*exp(-z**2*exp(2*u*z)/36 - 2/z - 8*exp(-u*z)/z)/(6*(4*exp(5*z**2/72) + exp(z**2/24) - 4*exp(z**2/18)))],
[(-(z**3*exp

Even though we could Finally get $\Phi$ of the original system, a limitation that we could optimize is its running time, yet nevertheless the computational ability has already surpassed human being.